# PMI-Based Phrase Detection for Inverted Index

**מטרה:** בניית אינדקס הפוך עם תמיכה בביטויים (phrases) באמצעות PMI

## מה זה PMI (Pointwise Mutual Information)?

PMI מודד את הסיכוי ששתי מילים מופיעות יחד בהשוואה לסיכוי שהן מופיעות בנפרד:

$$PMI(word_1, word_2) = \log_2 \frac{P(word_1, word_2)}{P(word_1) \cdot P(word_2)}$$

- **PMI גבוה** = המילים מופיעות יחד הרבה יותר ממה שהיינו מצפים → כנראה מושג אחד ("new york", "machine learning")
- **PMI נמוך/שלילי** = המילים מופיעות יחד כמו שהיינו מצפים או פחות

## שלב 1: התקנה ו-Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import math
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
# וודא ש-Spark עובד
spark

In [ ]:
# הגדרת ה-Bucket שלך
bucket_name = 'hw3ir322'  # שנה לשם ה-bucket שלך!
full_path = f"gs://{bucket_name}/"
paths = []

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if "parquet" in b.name:
        paths.append(full_path + b.name)

print(f"Found {len(paths)} parquet files")

## שלב 2: הגדרת Stopwords ו-Regex

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

print(f"Total stopwords: {len(all_stopwords)}")

## שלב 3: טעינת הנתונים

In [ ]:
# טעינת כל המסמכים מה-Parquet
parquetFile = spark.read.parquet(*paths)

# לבחירה: אינדקס על הכותרות או על הטקסט המלא
# עבור titles:
doc_text_pairs = parquetFile.select("title", "id").rdd

# עבור text (גוף המסמך):
# doc_text_pairs = parquetFile.select("text", "id").rdd

print(f"Total documents: {parquetFile.count()}")

## שלב 4: חישוב PMI לזיהוי ביטויים חזקים

### הלוגיקה:
1. ספירת כל המילים (unigrams)
2. ספירת כל צמדי המילים (bigrams)
3. חישוב PMI לכל bigram
4. סינון רק bigrams עם PMI גבוה ותדירות מספקת

In [ ]:
def extract_unigrams(text):
    """
    חילוץ כל המילים (unigrams) מטקסט
    """
    tokens = [token.group().lower() for token in RE_WORD.finditer(text)]
    # הסרת stopwords
    tokens = [t for t in tokens if t not in all_stopwords]
    return tokens


def extract_bigrams(text):
    """
    חילוץ כל צמדי המילים הסמוכים (bigrams) מטקסט
    """
    tokens = extract_unigrams(text)
    bigrams = []
    for i in range(len(tokens) - 1):
        bigrams.append((tokens[i], tokens[i+1]))
    return bigrams


def calculate_pmi(unigram_counts, bigram_counts, total_unigrams, total_bigrams):
    """
    חישוב PMI לכל bigram
    
    PMI(w1, w2) = log2( P(w1, w2) / (P(w1) * P(w2)) )
    
    Parameters:
    -----------
    unigram_counts: dict - מספר הופעות של כל מילה
    bigram_counts: dict - מספר הופעות של כל צמד
    total_unigrams: int - סה"כ מילים בקורפוס
    total_bigrams: int - סה"כ צמדים בקורפוס
    
    Returns:
    --------
    dict: bigram -> PMI score
    """
    pmi_scores = {}
    
    for (w1, w2), bigram_count in bigram_counts.items():
        # P(w1, w2) - הסתברות לראות את הצמד
        p_bigram = bigram_count / total_bigrams
        
        # P(w1), P(w2) - הסתברות לראות כל מילה בנפרד
        p_w1 = unigram_counts.get(w1, 0) / total_unigrams
        p_w2 = unigram_counts.get(w2, 0) / total_unigrams
        
        # מניעת חלוקה באפס
        if p_w1 > 0 and p_w2 > 0:
            pmi = math.log2(p_bigram / (p_w1 * p_w2))
            pmi_scores[(w1, w2)] = pmi
    
    return pmi_scores

In [ ]:
%%time
# שלב 4.1: ספירת Unigrams
print("Counting unigrams...")

# המרה לרשימת מילים וספירה
unigram_rdd = doc_text_pairs.flatMap(lambda x: extract_unigrams(x[0]))
unigram_counts_rdd = unigram_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

# איסוף לזיכרון
unigram_counts = dict(unigram_counts_rdd.collect())
total_unigrams = sum(unigram_counts.values())

print(f"Unique unigrams: {len(unigram_counts):,}")
print(f"Total unigrams: {total_unigrams:,}")

In [ ]:
%%time
# שלב 4.2: ספירת Bigrams
print("Counting bigrams...")

bigram_rdd = doc_text_pairs.flatMap(lambda x: extract_bigrams(x[0]))
bigram_counts_rdd = bigram_rdd.map(lambda bigram: (bigram, 1)).reduceByKey(lambda a, b: a + b)

# סינון bigrams עם תדירות מינימלית (לחסוך זיכרון)
MIN_BIGRAM_FREQ = 5  # bigram חייב להופיע לפחות 5 פעמים
filtered_bigram_counts_rdd = bigram_counts_rdd.filter(lambda x: x[1] >= MIN_BIGRAM_FREQ)

# איסוף לזיכרון
bigram_counts = dict(filtered_bigram_counts_rdd.collect())
total_bigrams = sum(bigram_counts.values())

print(f"Unique bigrams (freq >= {MIN_BIGRAM_FREQ}): {len(bigram_counts):,}")
print(f"Total bigrams: {total_bigrams:,}")

In [ ]:
%%time
# שלב 4.3: חישוב PMI
print("Calculating PMI scores...")

pmi_scores = calculate_pmi(unigram_counts, bigram_counts, total_unigrams, total_bigrams)

print(f"Calculated PMI for {len(pmi_scores):,} bigrams")

In [ ]:
# שלב 4.4: סינון ביטויים חזקים

# פרמטרים לכיול (אפשר לשחק עם הערכים)
PMI_THRESHOLD = 5.0        # PMI מינימלי - ערך גבוה = רק ביטויים "חזקים" מאוד
MIN_PHRASE_FREQ = 10       # תדירות מינימלית של הביטוי

# סינון לפי PMI ותדירות
strong_phrases = set()
for bigram, pmi in pmi_scores.items():
    if pmi >= PMI_THRESHOLD and bigram_counts.get(bigram, 0) >= MIN_PHRASE_FREQ:
        strong_phrases.add(bigram)

print(f"Strong phrases found: {len(strong_phrases):,}")

# הצגת דוגמאות
print("\nTop 30 phrases by PMI:")
sorted_phrases = sorted(
    [(bigram, pmi_scores[bigram], bigram_counts[bigram]) for bigram in strong_phrases],
    key=lambda x: x[1],
    reverse=True
)[:30]

for bigram, pmi, freq in sorted_phrases:
    print(f"  {bigram[0]}_{bigram[1]}: PMI={pmi:.2f}, freq={freq}")

In [ ]:
# שמירת הביטויים החזקים לקובץ pickle
phrases_filename = 'strong_phrases.pkl'

with open(phrases_filename, 'wb') as f:
    pickle.dump(strong_phrases, f)

print(f"Saved {len(strong_phrases):,} phrases to {phrases_filename}")

# העלאה ל-GCS
bucket = client.bucket(bucket_name)
blob = bucket.blob(f"phrases/{phrases_filename}")
blob.upload_from_filename(phrases_filename)

print(f"Uploaded to gs://{bucket_name}/phrases/{phrases_filename}")

## שלב 5: Tokenizer עם תמיכה בביטויים

הפונקציה הזו משמשת גם בבניית האינדקס וגם בזמן החיפוש

In [ ]:
def tokenize_with_phrases(text, strong_phrases, stopwords):
    """
    Tokenizer שמזהה ביטויים (bigrams חזקים) ומחבר אותם עם קו תחתון.
    
    לדוגמה: "New York City" → ["new_york", "city"]
    
    Parameters:
    -----------
    text: str - הטקסט לטוקניזציה
    strong_phrases: set of tuples - סט של (word1, word2) שמייצגים ביטויים
    stopwords: set - מילות עצירה
    
    Returns:
    --------
    list: רשימת טוקנים (כולל ביטויים מחוברים)
    """
    # חילוץ כל המילים
    all_tokens = [token.group().lower() for token in RE_WORD.finditer(text)]
    
    # הסרת stopwords
    tokens = [t for t in all_tokens if t not in stopwords]
    
    # עיבוד עם חלון נע לזיהוי ביטויים
    result = []
    i = 0
    while i < len(tokens):
        if i + 1 < len(tokens):
            bigram = (tokens[i], tokens[i+1])
            if bigram in strong_phrases:
                # מצאנו ביטוי! מחברים עם קו תחתון
                result.append(f"{tokens[i]}_{tokens[i+1]}")
                i += 2  # מדלגים על שתי המילים
                continue
        result.append(tokens[i])
        i += 1
    
    return result


# בדיקה
test_text = "New York is a great city in the United States"
test_result = tokenize_with_phrases(test_text, strong_phrases, all_stopwords)
print(f"Input: {test_text}")
print(f"Output: {test_result}")

## שלב 6: בניית ה-Inverted Index עם ביטויים

In [ ]:
# טעינת המודול של InvertedIndex
%cd -q /home/dataproc
!ls inverted_index_gcp.py

sc = spark.sparkContext
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0, SparkFiles.getRootDirectory())

from inverted_index_gcp import InvertedIndex

In [ ]:
# Broadcast של הביטויים לכל ה-Workers
strong_phrases_broadcast = sc.broadcast(strong_phrases)
all_stopwords_broadcast = sc.broadcast(all_stopwords)

print(f"Broadcasted {len(strong_phrases):,} phrases to all workers")

In [ ]:
NUM_BUCKETS = 124

def token2bucket_id(token):
    """ממפה טוקן למספר bucket"""
    return int(_hash(token), 16) % NUM_BUCKETS


def word_count_with_phrases(text, doc_id):
    """
    סופר תדירות של כל טוקן (כולל ביטויים) במסמך.
    
    Returns:
    --------
    list of tuples: [(token, (doc_id, tf)), ...]
    """
    tokens = tokenize_with_phrases(
        text, 
        strong_phrases_broadcast.value, 
        all_stopwords_broadcast.value
    )
    counts = Counter(tokens)
    return [(token, (doc_id, tf)) for token, tf in counts.items()]


def reduce_word_counts(unsorted_pl):
    """ממיין posting list לפי doc_id"""
    return sorted(unsorted_pl, key=lambda x: x[0])


def calculate_df(postings):
    """מחשב document frequency לכל טוקן"""
    return postings.map(lambda token: (token[0], len(token[1])))


def partition_postings_and_write(postings, base_dir):
    """מחלק ושומר את ה-posting lists"""
    bucket_rdd = postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey()
    
    def write_bucket(b_w_pl):
        bucket_id, word_posting_pairs = b_w_pl
        return InvertedIndex.write_a_posting_list(
            (bucket_id, list(word_posting_pairs)), base_dir, bucket_name
        )
    
    return bucket_rdd.map(write_bucket)

In [ ]:
%%time
# בניית האינדקס
print("Building inverted index with phrases...")

# שם התיקייה באחסון (שנה לפי הצורך)
INDEX_DIR = "title_with_phrases"  # או "text_with_phrases" לגוף הטקסט

# שלב 1: ספירת מילים
word_counts = doc_text_pairs.flatMap(lambda x: word_count_with_phrases(x[0], x[1]))

# שלב 2: יצירת posting lists
postings = word_counts.groupByKey().mapValues(reduce_word_counts)

# שלב 3: חישוב df
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()

print(f"Total unique tokens: {len(w2df_dict):,}")

In [ ]:
%%time
# כתיבת ה-posting lists
print(f"Writing posting lists to {INDEX_DIR}...")
_ = partition_postings_and_write(postings, INDEX_DIR).collect()
print("Done!")

In [ ]:
# איסוף כל מיקומי ה-posting lists
super_posting_locs = defaultdict(list)

for blob in client.list_blobs(bucket_name, prefix=INDEX_DIR):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

print(f"Collected posting locations for {len(super_posting_locs):,} tokens")

In [ ]:
# יצירה ושמירה של ה-InvertedIndex
inverted = InvertedIndex()
inverted.posting_locs = super_posting_locs
inverted.df = w2df_dict

# שמירה מקומית
inverted.write_index('.', 'index')

# העלאה ל-GCS
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/{INDEX_DIR}_idx/{index_src}'
!gsutil cp $index_src $index_dst

print(f"Index saved to {index_dst}")

## שלב 7: חישוב אורכי מסמכים (לשימוש ב-BM25)

In [ ]:
def calc_doc_length(doc_id, text):
    """מחשב אורך מסמך בטוקנים (כולל ביטויים)"""
    tokens = tokenize_with_phrases(
        text, 
        strong_phrases_broadcast.value, 
        all_stopwords_broadcast.value
    )
    return (doc_id, len(tokens))

# חישוב אורכים
doc_lengths_rdd = doc_text_pairs.map(lambda x: calc_doc_length(x[1], x[0]))
doc_lengths_dict = doc_lengths_rdd.collectAsMap()

print(f"Calculated lengths for {len(doc_lengths_dict):,} documents")

# שמירה
lengths_filename = 'doc_lengths.pickle'
with open(lengths_filename, 'wb') as f:
    pickle.dump(doc_lengths_dict, f)

# העלאה
lengths_dst = f'gs://{bucket_name}/{INDEX_DIR}_idx/{lengths_filename}'
!gsutil cp $lengths_filename $lengths_dst

print(f"Document lengths saved to {lengths_dst}")

## סיכום

יצרנו:
1. **`strong_phrases.pkl`** - סט של ביטויים חזקים לפי PMI
2. **`index.pkl`** - אינדקס הפוך עם תמיכה בביטויים
3. **`doc_lengths.pickle`** - אורכי מסמכים

### איך להשתמש ב-Frontend?

ב-`search_frontend.py` צריך:
```python
# טעינת הביטויים
with open('strong_phrases.pkl', 'rb') as f:
    strong_phrases = pickle.load(f)

# עיבוד השאילתה באותו אופן
def process_query(query):
    return tokenize_with_phrases(query, strong_phrases, all_stopwords)
```

## טיפים לכיול

| פרמטר | ערך נמוך | ערך גבוה |
|--------|----------|----------|
| `PMI_THRESHOLD` | הרבה ביטויים (כולל רעש) | רק ביטויים "ברורים" מאוד |
| `MIN_PHRASE_FREQ` | ביטויים נדירים נכללים | רק ביטויים נפוצים |
| `MIN_BIGRAM_FREQ` | יותר זיכרון, יותר מועמדים | פחות זיכרון, פחות מועמדים |

### ערכים מומלצים:
- **לכותרות (titles)**: PMI_THRESHOLD=4, MIN_PHRASE_FREQ=5
- **לגוף הטקסט (text)**: PMI_THRESHOLD=6, MIN_PHRASE_FREQ=20